<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/CNN/DCGAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, datasets, optimizers

import matplotlib.pyplot as plt
import sys
import numpy as np

In [2]:
class DCGAN():
  def __init__(self, rows, cols, channels, z=10):
    #input shape
    self.img_rows = rows
    self.img_cols = cols
    self.channels = channels
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.latent_dim = z

    optimizer = optimizers.Adam(0.0002, 0.5)
    
    #build and compile the discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    #build the generator
    self.generator = self.build_generator()

    #the generator takes noise as input and generates imgs
    z = tf.kears.Input(shape=(self.latent_dim,))
    img = self.generator(z)

    #생성기 판별기 결합
    self.combined = models.Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

  def build_generator(self):

    model = models.Sequential()

    model.add(layers.Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
    model.add(layers.Reshape((7, 7, 128)))
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(128, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation("relu"))
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(64, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation("relu"))
    model.add(layers.Conv2D(self.channels, kernel_size=3, padding="same"))
    model.add(layers.Activation("tanh"))

    model.summary()

    noise = tf.keras.Input(shape=(self.latent_dim,))
    img = model(noise)

    return Model(noise, img)

  def build_discriminator(self):

    model = models.Sequential()

    model.add(layers.Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(layers.ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    model.summary()

    img = tf.keras.Input(shape=self.img_shape)
    validity = model(img)

    return Model(img, validity)


